# State Farm Distracted Driver Detection - VGG16 Continued

This notebook contains the final attempt at the Kaggle State Farm Distracted Driver Detection competition using the VGG model extended with batchnorm. The purpose is to train the best possible model, as well as testing the external vgg16, utils and plot libraries.

## Initial Setup

Import libraries and functions for future use.

In [2]:
# Plots displayed inline in notebook
%matplotlib inline

# Make Python 3 consistent
from __future__ import print_function, division

# Make help libraries available
import sys

sys.path.append('/home/ubuntu/personal-libraries')

In [3]:
import numpy as np
import pandas as pd
import gc

from kerastools.vgg16 import Vgg16
from kerastools.utils import get_batches, save_array, load_array, get_classes, do_clip

from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, Flatten, Dropout
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


## Define model

We setup our initial VGG16 model with batchnormalisation

In [3]:
vgg = Vgg16(use_batchnorm = True)
vgg.model.summary()

552861696/553620808 [============================>.] - ETA: 0s_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_

## Setup batches

We define out validation and training badges for modelling

In [3]:
batch_size = 32

#path = ''
path = 'sample/'

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

NameError: name 'vgg' is not defined

## Finetune model - Sample

We need to adjust the standard VGG model to our new input with 10 classes, so we finetune it.

In [5]:
vgg.finetune(train_batches)
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

We train the model using the default learning rate of 0.001 for a single epoch

In [6]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
31/31 [==============================] - 26s - loss: 4.9359 - acc: 0.2137 - val_loss: 2.5414 - val_acc: 0.2708


We see that the accuracy increases fine on the sample, so we increase the learning rate.

In [7]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 25s - loss: 2.8378 - acc: 0.4265 - val_loss: 2.1342 - val_acc: 0.3824
Epoch 2/4
31/31 [==============================] - 25s - loss: 2.2904 - acc: 0.5263 - val_loss: 2.1866 - val_acc: 0.3971
Epoch 3/4
31/31 [==============================] - 25s - loss: 1.9057 - acc: 0.6039 - val_loss: 1.4286 - val_acc: 0.6176
Epoch 4/4
31/31 [==============================] - 26s - loss: 1.7159 - acc: 0.6210 - val_loss: 1.6112 - val_acc: 0.5521


Try 4 more epochs with lower learning rate.

In [8]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 25s - loss: 1.4559 - acc: 0.6808 - val_loss: 1.2820 - val_acc: 0.6618
Epoch 2/4
31/31 [==============================] - 25s - loss: 1.4853 - acc: 0.6805 - val_loss: 1.7984 - val_acc: 0.5441
Epoch 3/4
31/31 [==============================] - 25s - loss: 1.2413 - acc: 0.7311 - val_loss: 1.4632 - val_acc: 0.6176
Epoch 4/4
31/31 [==============================] - 25s - loss: 1.2351 - acc: 0.7137 - val_loss: 1.6869 - val_acc: 0.5147


Seems, this is as far as we can get on the sample data set. A pretty good base line in the area of 0.5 - 0.66.

## Finetune model - Full data

We continue our finetuning on the full data set.

In [9]:
path = ''

train_batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We start with a single epoch

In [10]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 546s - loss: 1.3370 - acc: 0.7291 - val_loss: 2.3063 - val_acc: 0.6103


We are going much better now. We increase the learning rate and see, where that takes us.

In [11]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 545s - loss: 1.0327 - acc: 0.7958 - val_loss: 2.2778 - val_acc: 0.6185
Epoch 2/4
613/613 [==============================] - 545s - loss: 1.0079 - acc: 0.8136 - val_loss: 2.9188 - val_acc: 0.5629
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.0285 - acc: 0.8199 - val_loss: 2.9374 - val_acc: 0.5730
Epoch 4/4
613/613 [==============================] - 546s - loss: 1.0548 - acc: 0.8284 - val_loss: 3.3877 - val_acc: 0.5513


And then we lower the learning rate again, and see where we end up.

In [12]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 1.0784 - acc: 0.8345 - val_loss: 3.1749 - val_acc: 0.5928
Epoch 2/4
613/613 [==============================] - 546s - loss: 1.0959 - acc: 0.8343 - val_loss: 2.9543 - val_acc: 0.6116
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.1056 - acc: 0.8427 - val_loss: 3.4353 - val_acc: 0.6040
Epoch 4/4
613/613 [==============================] - 545s - loss: 1.1361 - acc: 0.8414 - val_loss: 3.1795 - val_acc: 0.6048


The valuation accuracy is pretty good, but we are overfitting quite a lot. Lets try and make more layers trainable and see, if that helps things along.

In [13]:
layers = vgg.model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable = True

And then we rerun the training. First one epoch with low learning rate.

In [14]:
vgg.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 546s - loss: 1.1315 - acc: 0.8445 - val_loss: 3.5571 - val_acc: 0.6019


Then four epochs with a higher learning rate.

In [15]:
vgg.model.optimizer.lr = 0.1

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 545s - loss: 1.1549 - acc: 0.8485 - val_loss: 3.2152 - val_acc: 0.6196
Epoch 2/4
613/613 [==============================] - 545s - loss: 1.2350 - acc: 0.8449 - val_loss: 3.8607 - val_acc: 0.5636
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.1585 - acc: 0.8545 - val_loss: 3.1586 - val_acc: 0.6445
Epoch 4/4
613/613 [==============================] - 546s - loss: 1.1995 - acc: 0.8482 - val_loss: 3.3004 - val_acc: 0.6366


And then four epochs with a lower learning rate again.

In [16]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 1.1895 - acc: 0.8543 - val_loss: 3.7380 - val_acc: 0.6127
Epoch 2/4
613/613 [==============================] - 546s - loss: 1.1979 - acc: 0.8517 - val_loss: 3.7769 - val_acc: 0.5838
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.2458 - acc: 0.8541 - val_loss: 3.5832 - val_acc: 0.6203
Epoch 4/4
613/613 [==============================] - 545s - loss: 1.2328 - acc: 0.8580 - val_loss: 3.9601 - val_acc: 0.6012


That did little to improve things. Let's try an even lower learning rate.

In [17]:
vgg.model.optimizer.lr = 0.00001

vgg.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 546s - loss: 1.2667 - acc: 0.8545 - val_loss: 4.0369 - val_acc: 0.5770
Epoch 2/4
613/613 [==============================] - 546s - loss: 1.2413 - acc: 0.8590 - val_loss: 3.6576 - val_acc: 0.6109
Epoch 3/4
613/613 [==============================] - 545s - loss: 1.2565 - acc: 0.8591 - val_loss: 3.8979 - val_acc: 0.6073
Epoch 4/4
613/613 [==============================] - 545s - loss: 1.3083 - acc: 0.8600 - val_loss: 3.9093 - val_acc: 0.6062


We only seem to be stabilizing. Let's save the weights and try a different approach.

In [18]:
vgg.model.save_weights('models/base_vgg16_norm.h5')

## Improved VGG
We continue using the VGG16 network with batchnorm, but attempt to improve it. That is we want to keep the pretrained convolutional layers fixed, and increase our training speed.
We start by defining a new VGG16() model.

In [4]:
vgg = Vgg16(use_batchnorm = True)
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

We then proceed to find the last max pooling layer of the model.

In [5]:
# Define convolutional layers
last_conv_idx = [i for i, l in enumerate(vgg.model.layers) if type(l) is MaxPooling2D][-1]
conv_layers = vgg.model.layers[:last_conv_idx + 1]

We can then define a model using only the convolutional layers.

In [6]:
conv_model = Sequential(conv_layers)
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
__________

The idea is now, that we want to pre-computer all of our data through the convolutional layers. This will drastically reduce the training time, once we start experimenting with dense model architecture.
We start by defining our batches.

In [7]:
path = ''

train_batches = get_batches(path + 'train', batch_size = 44, target_size = (224, 224), shuffle = False)

valid_batches = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False)

test_batches = get_batches(path + 'test', batch_size = 2, target_size = (224, 224), shuffle = False, class_mode = None)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We also extract labels and classes for each dataset.

In [8]:
(val_classes, trn_classes, val_labels, trn_labels, 
 val_filenames, filenames, test_filenames) = get_classes(path)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


We then pre-compute each of our datasets and save the numpy arrays. This eats a lot of memory on the poor AWS instance, so after each data computation, we do some cleanup to realease the memory. We save and load using bcolz, as it utilises great compression and does I/O very fast.

In [12]:
conv_feat = conv_model.predict_generator(train_batches, np.int(train_batches.samples / train_batches.batch_size))
save_array(path + 'results/conv_computed/conv_feat_norm.dat', conv_feat)

del conv_feat
gc.collect()

0

In [13]:
conv_val_feat = conv_model.predict_generator(valid_batches, np.int(valid_batches.samples / valid_batches.batch_size))
save_array(path + 'results/conv_computed/conv_val_feat_norm.dat', conv_val_feat)

del conv_val_feat
gc.collect()

0

In [14]:
conv_test_feat = conv_model.predict_generator(test_batches, np.int(test_batches.samples / test_batches.batch_size))
save_array(path + 'results/conv_computed/conv_test_feat_norm.dat', conv_test_feat)

del conv_test_feat
gc.collect()

0

And finally, we can load the three feature sets

In [15]:
conv_feat = load_array('results/conv_computed/conv_feat_norm.dat')
conv_val_feat = load_array('results/conv_computed/conv_val_feat_norm.dat')
conv_test_feat = load_array('results/conv_computed/conv_test_feat_norm.dat')

## VGG model with batchnorm and precomputed augmentation.

We precompute some augmented data in order to reduce the overfitting of out model. We start by setting the level of preprocessing and then define a new bunch of batches.

We then train the model on the augmented images. First define a new model.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
norm_layer (Lambda)          (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_40 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv_layer_1_0 (Conv2D)      (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv_layer_1_1 (Conv2D)      (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 112, 112)      0         
__________

Define batches

In [23]:
gen_t = image.ImageDataGenerator(rotation_range = 15,
                                 height_shift_range = 0.05,
                                 shear_range = 0.1,
                                 channel_shift_range = 20,
                                 width_shift_range = 0.1)
path = ''

da_batches = vgg.get_batches(path + 'train',
                             gen_t,
                             batch_size = 44,
                             shuffle = True,
                             target_size = (224, 224))
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


Compile model

In [24]:
vgg.compile()

Finetune and make all dense layers trainable

In [25]:
vgg.finetune(da_batches)

layers = vgg.model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable = True

Then train the model using the augmented batches. First run some epochs at default learning rate.

In [26]:
vgg.fit_batch(da_batches, val_batches, 4)

Epoch 1/4
446/446 [==============================] - 545s - loss: 3.0292 - acc: 0.4316 - val_loss: 1.5342 - val_acc: 0.5453
Epoch 2/4
446/446 [==============================] - 544s - loss: 2.2398 - acc: 0.5530 - val_loss: 1.6148 - val_acc: 0.5495
Epoch 3/4
446/446 [==============================] - 544s - loss: 2.1429 - acc: 0.5811 - val_loss: 1.6315 - val_acc: 0.5766
Epoch 4/4
446/446 [==============================] - 544s - loss: 2.1489 - acc: 0.5919 - val_loss: 1.6767 - val_acc: 0.6051


Then increase the learning rate and run some more epochs

In [29]:
vgg.model.optimizer.lr = 0.01

vgg.fit_batch(da_batches, val_batches, 4)

Epoch 1/4
 40/446 [=>............................] - ETA: 440s - loss: 2.3440 - acc: 0.5835

KeyboardInterrupt: 

Finally decrease the learning rate and run some more epochs.

In [ ]:
vgg.model.optimizer.lr = 0.001

vgg.fit_batch(train_batches, val_batches, 4)

## Pseudolabeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. Afterwards we add the test set as well.

In [22]:
val_pseudo = vgg.predict(val_batches, batch_size = 50)

We concatenate thse pseudo labels with our training labels

In [24]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

And train our model using the extended data set.

In [26]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/1
120544/120544 [==============================] - 49s - loss: 0.3544 - acc: 0.8902 - val_loss: 0.4987 - val_acc: 0.8382


We do not really see much of an improvement. Let's try 4 more epochs.

In [27]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 50s - loss: 0.3426 - acc: 0.8951 - val_loss: 0.4796 - val_acc: 0.8421
Epoch 2/4
120544/120544 [==============================] - 50s - loss: 0.3360 - acc: 0.8967 - val_loss: 0.5171 - val_acc: 0.8375
Epoch 3/4
120544/120544 [==============================] - 50s - loss: 0.3269 - acc: 0.8995 - val_loss: 0.5132 - val_acc: 0.8371
Epoch 4/4
120544/120544 [==============================] - 50s - loss: 0.3224 - acc: 0.9010 - val_loss: 0.5321 - val_acc: 0.8364


Now we are crossing the 0.9 threshold of accuracy. Lets lower the learning rate and train for 4 more epochs and see where that gets us.

In [29]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 120544 samples, validate on 2800 samples
Epoch 1/4
120544/120544 [==============================] - 50s - loss: 0.3134 - acc: 0.9038 - val_loss: 0.5191 - val_acc: 0.8411
Epoch 2/4
120544/120544 [==============================] - 50s - loss: 0.3062 - acc: 0.9060 - val_loss: 0.5285 - val_acc: 0.8414
Epoch 3/4
120544/120544 [==============================] - 50s - loss: 0.3033 - acc: 0.9071 - val_loss: 0.4933 - val_acc: 0.8443
Epoch 4/4
120544/120544 [==============================] - 50s - loss: 0.2933 - acc: 0.9103 - val_loss: 0.4806 - val_acc: 0.8475


So we do see a pretty nice improvement. Enough to warrent us trying with the entire test set.

In [30]:
test_pseudo = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

We concatenate thse pseudo labels with our training and valuation pseudo labels

In [31]:
comb_pseudo = np.concatenate([comb_pseudo, test_pseudo])
comb_feat = np.concatenate([comb_feat, conv_test_feat])

And train our model using the extended data set.

In [32]:
bn_model_bigger.optimizer.lr = 0.001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 1,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/1
200270/200270 [==============================] - 84s - loss: 0.5545 - acc: 0.8444 - val_loss: 0.4587 - val_acc: 0.8454


Hrm, too early to say, but valuation accuracy holds still. Let's run 4 more epochs.

In [33]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 84s - loss: 0.5158 - acc: 0.8610 - val_loss: 0.4763 - val_acc: 0.8346
Epoch 2/4
200270/200270 [==============================] - 83s - loss: 0.4985 - acc: 0.8692 - val_loss: 0.4769 - val_acc: 0.8339
Epoch 3/4
200270/200270 [==============================] - 83s - loss: 0.4936 - acc: 0.8704 - val_loss: 0.4463 - val_acc: 0.8493
Epoch 4/4
200270/200270 [==============================] - 84s - loss: 0.4850 - acc: 0.8746 - val_loss: 0.4865 - val_acc: 0.8396


Let's lower the learning rate again and run 4 more epochs.

In [34]:
bn_model_bigger.optimizer.lr = 0.00001

bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 4,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/4
200270/200270 [==============================] - 83s - loss: 0.4803 - acc: 0.8757 - val_loss: 0.4721 - val_acc: 0.8389
Epoch 2/4
200270/200270 [==============================] - 84s - loss: 0.4764 - acc: 0.8771 - val_loss: 0.4542 - val_acc: 0.8464
Epoch 3/4
200270/200270 [==============================] - 83s - loss: 0.4750 - acc: 0.8788 - val_loss: 0.4759 - val_acc: 0.8396
Epoch 4/4
200270/200270 [==============================] - 83s - loss: 0.4700 - acc: 0.8799 - val_loss: 0.4844 - val_acc: 0.8364


We are improving slowly. Let's run for 10 more epochs.

In [36]:
bn_model_bigger.fit(x = comb_feat,
                    y = comb_pseudo,
                    batch_size = batch_size,
                    epochs = 10,
                    validation_data = (conv_val_feat, val_labels))

Train on 200270 samples, validate on 2800 samples
Epoch 1/10
200270/200270 [==============================] - 83s - loss: 0.4671 - acc: 0.8815 - val_loss: 0.4400 - val_acc: 0.8461
Epoch 2/10
200270/200270 [==============================] - 83s - loss: 0.4642 - acc: 0.8820 - val_loss: 0.5003 - val_acc: 0.8371
Epoch 3/10
200270/200270 [==============================] - 83s - loss: 0.4617 - acc: 0.8831 - val_loss: 0.5024 - val_acc: 0.8386
Epoch 4/10
200270/200270 [==============================] - 84s - loss: 0.4578 - acc: 0.8849 - val_loss: 0.5042 - val_acc: 0.8339
Epoch 5/10
200270/200270 [==============================] - 83s - loss: 0.4557 - acc: 0.8858 - val_loss: 0.4949 - val_acc: 0.8368
Epoch 6/10
200270/200270 [==============================] - 83s - loss: 0.4552 - acc: 0.8843 - val_loss: 0.4806 - val_acc: 0.8379
Epoch 7/10
200270/200270 [==============================] - 83s - loss: 0.4532 - acc: 0.8865 - val_loss: 0.4609 - val_acc: 0.8436
Epoch 8/10
200270/200270 [==============

Having trained the model, we save the weights.

In [37]:
bn_model_bigger.save_weights('models/batchnorm_vgg16.h5')

## Submitting to Kaggle

We finally submit the improved model to Kaggle.

We start by finding the optimal level of clipping.

In [48]:
valid_batches_pred = get_batches(path + 'valid', batch_size = 50, target_size = (224, 224), shuffle = False, class_mode = None)
conv_val_feat_pred = conv_model.predict_generator(valid_batches_pred, np.int(valid_batches_pred.samples / valid_batches_pred.batch_size))

val_predictions = bn_model_bigger.predict(conv_val_feat_pred, batch_size = 50)

Found 2800 images belonging to 10 classes.


In [57]:
def do_clip(arr, mx): return np.clip(arr, (1 - mx) / 9, mx)

We then proceed to determine the optimal level of clipping using the validation data set.

In [58]:
test_clip = []
for i in np.arange(0.70, 1.0, 0.01):
    test_clip.append([i, categorical_crossentropy(val_labels, do_clip(val_predictions, i)).eval().mean()])

min(test_clip, key = lambda x: x[1])

[1.0000000000000002, 0.48164355679327142]

Here it is said, that no clipping is best. Weird. Lets submit two. One clipped and one not clipped. First we compute the predictions.

In [59]:
test_predictions = bn_model_bigger.predict(conv_test_feat, batch_size = 2)

Define classes

In [61]:
classes = sorted(valid_batches.class_indices, key = valid_batches.class_indices.get)

Then we make a cliping based on earlier experience

In [62]:
sumbit_pred = do_clip(test_predictions, 0.89)

Then we prepare a submission without clipping.

In [65]:
submission_no_clip = pd.DataFrame(test_predictions, columns = classes)
submission_no_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_no_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.006723,0.004307,0.000682,0.001594,0.001567,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.007839,0.000621,0.002267,0.001608,0.003189,0.001825,0.000393,0.005292,0.287327
2,img_62885.jpg,0.005301,0.000154,0.000257,0.017661,0.973055,0.000493,0.000514,0.000016,0.000978,0.001571
3,img_45125.jpg,0.002003,0.007311,0.017353,0.000462,0.002328,0.000509,0.656182,0.006239,0.302687,0.004926
4,img_22633.jpg,0.138601,0.055678,0.011123,0.001665,0.004929,0.012565,0.022692,0.007558,0.186375,0.558814


And a submission with clipping

In [66]:
submission_clip = pd.DataFrame(sumbit_pred, columns = classes)
submission_clip.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission_clip.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.046099,0.012222,0.012222,0.012222,0.012222,0.012222,0.026646,0.020867,0.016716,0.874798
1,img_14887.jpg,0.689640,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.012222,0.287327
2,img_62885.jpg,0.012222,0.012222,0.012222,0.017661,0.890000,0.012222,0.012222,0.012222,0.012222,0.012222
3,img_45125.jpg,0.012222,0.012222,0.017353,0.012222,0.012222,0.012222,0.656182,0.012222,0.302687,0.012222
4,img_22633.jpg,0.138601,0.055678,0.012222,0.012222,0.012222,0.012565,0.022692,0.012222,0.186375,0.558814


Finally we save the two submissions

In [67]:
submission_file_name_no_clip = 'results/augmented-pseudo-vgg-no-clip.gz'
submission_no_clip.to_csv(submission_file_name_no_clip, index = False, compression = 'gzip')

submission_file_name_clip = 'results/augmented-pseudo-vgg-clip.gz'
submission_clip.to_csv(submission_file_name_clip, index = False, compression = 'gzip')

In [70]:
from IPython.display import FileLink
FileLink('results/augmented-pseudo-vgg-no-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-no-clip.gz

In [71]:
FileLink('results/augmented-pseudo-vgg-clip.gz')

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/results/augmented-pseudo-vgg-clip.gz

Turns out that in this case, the no clipping submission actually performed best, by a absolute 0.03.